In [8]:
import cv2

In [9]:
import PIL

In [15]:
import cv2
face_detect=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')   #face is extracted using this
video=cv2.VideoCapture(0)    #capture image from webcam(if web cam is connected separately use 1 in place of 0)
id=input('Enter user id')    #enter any id(should be unique)
sampleNum=0                  
while True:
    check,frame=video.read()         #reads the face 
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)            #changing coloured image to gray scale image(B&W)
    face=face_detect.detectMultiScale(gray,scaleFactor=1.05,minNeighbors=15)    #detect the multiscale image by detecting frames
    for (x,y,w,h) in face:                   #extract the faces and store it with your name
        sampleNum=sampleNum+1
        cv2.imwrite('Dataset//User.'+str(id)+"."+str(sampleNum)+".jpg",gray[y:y+h,x:x+w])   #path where it should be stored
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        cv2.waitKey(100);
    cv2.imshow("Vishnu",frame)
    cv2.waitKey(1);
    if(sampleNum>30):
        break
video.release()
cv2.destroyAllWindows()    #closes after taking 30 images of the face

Enter user id2


In [16]:
import cv2,os
import numpy as np
from PIL import Image    
#we are training the face saple here by putting all the captured image pixels into an array for this the PIL is used
recognizer = cv2.face.LBPHFaceRecognizer_create()    #localBinaryPattern(LBP) and H stands for histogram(extracts pixels from images and puts them into histogram)
detector= cv2.CascadeClassifier('haarcascade_frontalface_default.xml');
path=("Dataset")

def getImagesAndLabels(path):
    #get the path of all the files in the folder
    imagePaths=[os.path.join(path,f) for f in os.listdir(path)] 
    #create empth face list
    faceSamples=[]
    #create empty ID list
    Ids=[]
    #now looping through all the image paths and loading the Ids and the images
    for imagePath in imagePaths:
        #loading the image and converting it to gray scale
        pilImage=Image.open(imagePath).convert('L')
        #Now we are converting the PIL image into numpy array
        imageNp=np.array(pilImage,'uint8')
        #getting the Id from the image
        Id=int(os.path.split(imagePath)[-1].split(".")[1])
        # extract the face from the training image sample
        faces=detector.detectMultiScale(imageNp)
        #If a face is there then append that in the list as well as Id of it
        for (x,y,w,h) in faces:
            faceSamples.append(imageNp[y:y+h,x:x+w])
            Ids.append(Id)
    return faceSamples,Ids


faces,Ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(Ids))
recognizer.save('Recogniser\\trainner.yml')
cv2.destroyAllWindows()


In [17]:
#this is the final step and here we are dtecting the faces
import cv2
face_detect=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
video=cv2.VideoCapture(0)
rec=cv2.face.LBPHFaceRecognizer_create()
rec.read("Recogniser\\trainner.yml")
         
id=0
font=cv2.FONT_HERSHEY_SIMPLEX          #font style

while True:
    check,frame=video.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)         #converts the colour image to grayscale to detect 
    face=face_detect.detectMultiScale(gray,scaleFactor=1.05,minNeighbors=15)
    for (x,y,w,h) in face:
        cv2.rectangle(frame,(x,y),(x+w++3,y+h++3),(0,255,0),3)       #create rectangle frame
        id,conf=rec.predict(gray[y:y+h,x:x+w])
        if (id==1):
            id='Vishnu'
        #elif id==2:
            #id='someone'
        cv2.putText(frame,str(id),(x+h,y),font,1,(255,0,0),2);
    cv2.imshow("Vishnu",frame)
    key=cv2.waitKey(1)
    if key==ord('k'):
        break
video.release()
cv2.destroyAllWindows()
